# Example usage

This is an example tutorial of how to `tdads` can be used to analyze persistence diagrams. First, let's make sure we have all the necessary dependencies:

In [1]:
%pip install ripser
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Now we'll import all the packages we'll need:

In [8]:
from tdads.machine_learning import *
from tdads.inference import *
from numpy.random import uniform
from numpy import array
from math import cos, sin, pi
from ripser import ripser
import matplotlib.pyplot as plt

For this tutorial we'll create helper functions for generating data and persistence diagrams from circle and spheres:

In [9]:
# function to create a circle dataset and
# compute its diagram
def circle_diagram():
    # sample 100 points from the unit circle
    theta = uniform(low = 0, high = 2*pi, size = 100)
    data = array([[cos(theta[i]), sin(theta[i])] for i in range(100)])
    # compute persistence diagram
    diag = ripser(data, maxdim = 2)
    return [data, diag]

# function to create a sphere dataset and
# compute its diagram
def sphere_diagram():
    # sample 100 points from the unit sphere
    phi = uniform(low = 0, high = 2*pi, size = 100)
    theta = uniform(low = 0, high = pi, size = 100)
    data = array([[sin(theta[i])*cos(phi[i]), sin(theta[i])*sin(phi[i]), cos(theta[i])] for i in range(100)])
    # compute persistence diagram
    diag = ripser(data, maxdim = 2)
    return [data, diag]

Our dataset will be 5 circle datasets (and diagrams) and 5 sphere datasets (and diagrams), totalling 10:

In [10]:
result = [circle_diagram(), circle_diagram(), circle_diagram(), circle_diagram(), circle_diagram(),
          sphere_diagram(), sphere_diagram(), sphere_diagram(), sphere_diagram(), sphere_diagram()]
data = [r[0] for r in result]
diagrams = [r[1] for r in result]

Now we will use the bootstrap procedure to determine the significant topological features in each diagram:

In [11]:
def diag_fun(X, thresh):
    return ripser(X = X, thresh = thresh, maxdim = 2)
boot = diagram_bootstrap(diag_fun = diag_fun, dims = [0,1,2], alpha = 0.01)
boot_diagrams = [boot.compute(X = d, thresh = 2) for d in data]

The subsetted diagrams show that only the first five diagrams have one loop and only the last five diagrams have one void:

In [12]:
for i in range(10):
    print('Num clusters:' + str(len(boot_diagrams[i]['subsetted_diagram'][0])) + ', num loops: ' + str(len(boot_diagrams[i]['subsetted_diagram'][1])) + ', num voids: ' + str(len(boot_diagrams[i]['subsetted_diagram'][2])))

Num clusters:2, num loops: 1, num voids: 5
Num clusters:1, num loops: 1, num voids: 3
Num clusters:1, num loops: 1, num voids: 3
Num clusters:1, num loops: 1, num voids: 3
Num clusters:2, num loops: 1, num voids: 2
Num clusters:4, num loops: 3, num voids: 1
Num clusters:22, num loops: 1, num voids: 1
Num clusters:23, num loops: 2, num voids: 1
Num clusters:5, num loops: 2, num voids: 1
Num clusters:14, num loops: 1, num voids: 1


It seems like we have resolved the two groups of persistence diagrams, but a 2D MDS projection of the 10 diagrams visually confirms as much:

In [13]:
mds = diagram_mds(p = float('inf'), dim = 2) # for 2-dimensional homology
emb = mds.fit_transform(diagrams)
plt.scatter(emb[:,0], emb[:,1], color = ['red','red','red','red','red','blue','blue','blue','blue','blue'])
plt.xlabel('Embedding dim 1')
plt.ylabel('Embedding dim 2')
plt.show()

ValueError: Array must be symmetric

Finally we can use a permutation test of our two suspected groups to statistically capture their differences in dimensions 0, 1 and 2 (all are near/below 0.05):

In [ ]:
pt = perm_test(p = float('inf'), iterations = 50, dims = [0,1,2])
res = pt.test([[d for d in diagrams[0:5]], [d for d in diagrams[5:10]]])
res['p_values']